# CC pipeline: Run the StarDist2D segmentation model over all folders

This notebook loads a pretrained StarDist2D segmentation model and applies the segmentation prediction on all folders within the masterfolder mainf (defined in 2nd code cell). Only microscopy chamber data containing folders should be within mainf. The segmentation is applied onto all images that end with *_PH.tif* and the segmentation image is saved into a newly created folder within each image folder named *seg_sd2*. For the moment, it assumes single-page tif files and saves single-page tif files with the exact same name as the input image used for segmentation prediction.

In [1]:
# directory
configdir = 'D://GitHub/azimages/julian/CC_pipeline/'# name of config file
mainconfigname = 'jbanalysisconfig_cc.json';

if not mainconfigname.endswith('.json'):
    mainconfigname += '.json'

import json
# Read JSON data
with open(configdir+mainconfigname, 'r') as file:
    data = json.load(file)

# Assign each key-value pair as a variable
for key, value in data.items():
    globals()[key] = value

In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from tifffile import imread, imwrite
from datetime import datetime
from csbdeep.utils import Path, normalize
from skimage.measure import regionprops, regionprops_table
from skimage import io
from skimage import segmentation
from skimage import color
from stardist.matching import matching_dataset
from stardist import fill_label_holes, random_label_cmap, relabel_image_stardist, calculate_extents, gputools_available, _draw_polygons
from stardist.models import Config2D, StarDist2D, StarDistData2D
import os
from tensorflow import keras
import cv2
import pandas as pd
from datetime import date
np.random.seed(42)
lbl_cmap = random_label_cmap()

def add_prefix(props, prefix):
    return {f"{prefix}_{key}" if 'intensity' in key else key: value for key, value in props.items()}

### Check if GPU can be accessed

In [3]:
gputools_available()

If you want to compute separable approximations, please install it with
pip install scikit-tensor-py3


True

### Load in meta file and display head. Check if correct

In [13]:
meta = pd.read_csv(os.path.join(masterdir,metacsv), dtype={'stardist': str})
replicates = meta.replicate.unique()
meta.head()

,dynamic_class,combination,OriginalFolder,replicate,TopStrain,BotStrain,type,media,flow,MaxFr,...,Note,StageX,StageY,PxinUmX,PxinUmY,BarrierYincrop,chamberbox1,chamberbox2,chamberbox3,chamberbox4
0,switched dominance,agr-I vs agr-III,E:\Julian\barrier\Folder20221117_20221116_barr...,b02,MW2,LAC,Compete,TSB50,0.25,NaN,...,NaN,-63515.595160,-37742.727670,0.065,0.065,760,527,347,713,1467
1,unstable dual activation,agr-I vs agr-II,E:\Julian\barrier\Folder20221118_20221117_barr...,b03,LAC,502a,Compete,TSB50,0.25,250.0,...,NaN,-55537.130050,-39137.541060,0.065,0.065,753,593,323,747,1467
2,delayed dual activation,agr-I vs agr-II,E:\Julian\barrier\Folder20230212_20230125_muli...,b14,LAC,502a,Compete,TSB50,0.25,105.0,...,NaN,-59892.382803,-34289.970562,0.065,0.065,713,820,377,740,1467
3,initial dominance,agr-I vs agr-IV,E:\Julian\barrier\Folder20230215_20230213_muli...,b16,MNTG,LAC,Compete,TSB50,0.25,120.0,...,NaN,-58403.877410,-38595.147860,0.065,0.065,773,573,387,707,1467
4,stable dominance,agr-I vs agr-III,E:\Julian\barrier\Folder20230128_20230125_muli...,b19,MW2,LAC,Compete,TSB50,0.25,NaN,...,NaN,-47656.482690,-34944.612790,0.065,0.065,720,707,283,747,1467


### Load stardist model
Here, the model is loaded. You need to specify the dir which contains a folder named *stardist* in the config file. This *stardist* folder needs to contain the files *weigths_best.h5* as well as the *config.json* and optionally the *thresholds.json*

In [14]:
print(stardistmodeldir)
model = StarDist2D(None, name='stardist', basedir=stardistmodeldir)
axis_norm = (0,1)   # normalize channels independently

D://GitHub/azimages/julian/stardist/models/stardist-bar
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.493779, nms_thresh=0.3.


### Define regionprops parameters. You could add more if you want to

In [15]:
if flims:
    prop_list = ['label', 
                'area', 'centroid', 
                'axis_major_length', 'axis_minor_length',
                 'eccentricity',
                'intensity_mean', 'intensity_max']
else:
    prop_list = ['label', 
                'area', 'centroid', 
                'axis_major_length', 'axis_minor_length',
                 'eccentricity'] 

### Limit GPU RAM usage by StarDist

In [16]:
from csbdeep.utils.tf import limit_gpu_memory
# adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
limit_gpu_memory(fraction=ramlimit, total_memory=ramsize)
# alternatively, try this:
# limit_gpu_memory(None, allow_growth=True)

Virtual devices cannot be modified after being initialized


## Main segmentation loop
This loop goes over each row in the meta file which is marked with completed preprocessing (Progress == 'Done') and applies the StarDist segmentation model to each position/chamber iteratively. For the moment, not paralellized but could probably benefit from that.

In [17]:
for i in range(meta.shape[0]):
    # Skip if already processed or marked for exclusion
    if meta.loc[i, 'stardist'] == 'Done' or meta.loc[i, 'Exclude'] == 'excl':
        continue

    # Directory setup for current experiment
    main_folder = os.path.join(masterdir, savedirname, meta.replicate[i], 'Chambers')
    save_directory = os.path.join(main_folder, 'stardistdata')
    os.makedirs(save_directory, exist_ok=True)

    current_directory = os.path.join(main_folder, f'Pos{str(meta.pos[i]).zfill(2)}_Chamb01')
    if not os.path.exists(current_directory):
        continue

    output_folder = os.path.join(current_directory, "seg_sd2")
    os.makedirs(output_folder, exist_ok=True)
    # Clear output folder if not empty
    for file in Path(output_folder).glob('*tif'):
        os.remove(file)

    # Collecting timelapse images
    images = sorted(Path(current_directory).glob('*Ch1*tif'))
    # Additional channels if applicable
    if flims:
        images_fl = sorted(Path(current_directory).glob('*Ch2*tif'))
        if n_channel > 2:
            images_fl2 = sorted(Path(current_directory).glob('*Ch3*tif'))

    # Determine frame range
    max_frame = meta.loc[i, 'MaxFr']
    frame_list = range(len(images)) if np.isnan(max_frame) else range(int(max_frame) - 1)
    
    # Process each frame
    fails = []  # To record any failures
    for frame_index in tqdm(frame_list, desc=f"{meta.replicate[i]}, pos {str(meta.pos[i]).zfill(2)}"):
        try:
            # Reading fluorescence images if available
            if flims:
                fluorescence_image = imread(images_fl[frame_index])
                if n_channel > 2:
                    fluorescence_image2 = imread(images_fl2[frame_index])
            
            # Main segmentation process
            main_image = imread(images[frame_index])
            normalized_image = normalize(main_image, 1, 99.8, axis=axis_norm)
            labels, details = model.predict_instances(normalized_image)
            
            # Save segmentation labels
            filename_segmentation = os.path.join(output_folder, os.path.basename(images[frame_index]))
            imwrite(filename_segmentation, labels, append=False, metadata=None)
            
            # Region properties calculation
            region_props = regionprops_table(labels, intensity_image=fluorescence_image if flims else None, properties=prop_list)
            if flims and n_channel > 2:
                region_props = add_prefix(region_props, 'fluor1')
                region_props2 = regionprops_table(labels, intensity_image=fluorescence_image2, properties=prop_list)
                region_props2 = add_prefix(region_props2, 'fluor2')
                # Merge intensity data for multiple fluorescence channels
                for key, value in region_props2.items():
                    if 'intensity' in key:
                        region_props[key] = value
            
            # Dataframe handling: compile and format region properties
            region_props_df = pd.DataFrame(region_props)
            region_props_df.insert(0, 'frame', frame_index + 1)
            region_props_df.insert(1, 'pos', int(os.path.basename(current_directory)[-2:]))
            region_props_df.insert(2, 'replicate', meta.replicate[i])
            region_props_df['folder'] = current_directory
            
            if frame_index == 0:
                all_frames_df = region_props_df
            else:
                all_frames_df = pd.concat([all_frames_df, region_props_df], ignore_index=True)
            
        except Exception as e:
            # Log any errors encountered during processing
            fails.append(f"Error processing folder {current_directory}, Frame {frame_index}: {e}")

    # Save compiled data for current position
    if 'all_frames_df' in locals():
        all_frames_df.to_csv(os.path.join(save_directory, os.path.basename(current_directory) + '.csv'), index=False)

    # Update metadata to indicate completion and log any failures
    meta = pd.read_csv(masterdir+metacsv, dtype={'stardist': str})
    meta.at[i, 'stardist'] = 'Done'
    if fails:
        meta.at[i, 'stardist_fails'] = '; '.join(fails)

    # Save metadata with updates
    meta.to_csv(os.path.join(masterdir, metacsv), index=False)

b14, pos 30: 100%|███████████████████████████████████████████████████████████████████| 104/104 [04:26<00:00,  2.56s/it]


In [ ]:
for i in range(0, meta.shape[0]):
    fails = []
    if  not meta.loc[i, ('stardist')] == 'Done':
        if meta.loc[i, ('Exclude')] == 'excl':
            continue
            
        fails = []

        # directory variables
        mainf = masterdir + savedirname + '/' + meta.replicate[i] + '/Chambers'
        savedir = mainf + '/' + 'stardistdata' + '/'
        if not os.path.exists(savedir):
            os.makedirs(savedir)
        currd = mainf + '/Pos' + str(meta.pos[i]).zfill(2) + '_Chamb01'
        if not os.path.exists(currd):
            continue      
        inputs_folder = currd
        outputs_folder = os.path.join(inputs_folder, "seg_sd2")
        if not os.path.exists(outputs_folder):
                os.makedirs(outputs_folder)
        else:
            filestodel = sorted(Path(outputs_folder).glob('*tif'))
            for f in filestodel:
                os.remove(f)
            del filestodel

        # list of all images per channel
        fXt = sorted(Path(inputs_folder).glob('*Ch1*tif'))
        if flims:
            fXtfl = sorted(Path(inputs_folder).glob('*Ch2*tif'))
            if n_channel>2:
                fXtfl2 = sorted(Path(inputs_folder).glob('*Ch3*tif'))

        maxfr = meta.loc[i, ('MaxFr')]
        if np.isnan(maxfr):
            imlist = range(0, len(fXt))
        else:
            imlist = range(0, (maxfr.astype(int)-1))
        
        for imi in tqdm(imlist, desc=meta.replicate[i] + ', pos ' + str(meta.pos[i]).zfill(2)):
            try:
                # read in fluor images
                if flims:
                    flim = imread(fXtfl[imi])
                    if n_channel>2:
                        flim2 = imread(fXtfl2[imi])
                # main segmentation images
                ims = imread(fXt[imi])
                # label image will have same name as Ch1 image
                filenameseg = os.path.join(outputs_folder, os.path.basename(fXt[imi]))

                # normalize
                img = normalize(ims, 1,99.8, axis=axis_norm)
                del ims
                
                # segmentation
                labels, details = model.predict_instances(img)
                del img
                
                #write label image
                imwrite(filenameseg,
                             labels,
                             append=False, metadata=None)
                
                #regionprops measurements
                if flims:
                    reg_props = regionprops_table(labels, intensity_image=flim, properties=prop_list)
                    #if more than 1 fluor channel, do a 2nd regionprops and add a prefix of fluor1 and fluor2
                    if n_channel>2:
                        reg_props = add_prefix(reg_props, 'fluor1')
                        reg_props2 = regionprops_table(labels, intensity_image=flim2, properties=prop_list)
                        reg_props2 = add_prefix(reg_props2, 'fluor2')
                        # Add only the 'intensity' containing keys from props_flim2_prefixed
                        for key, value in reg_props2.items():
                            if 'intensity' in key:
                                reg_props[key] = value
                else:
                    reg_props = regionprops_table(labels,  properties=prop_list)
                    
                #dateframe handling: format the regionprops into pandas, add frame, chamber, position, replicate and folder info
                # if 1st frame, reset. Otherwise append
                if imi == 0:
                    if 'df' in locals():
                        del df
                    df = pd.DataFrame(reg_props)
                    df.insert(0,'frame', imi+1)
                    df.insert(0,'pos', int((os.path.basename(currd)[-2:])))
                    df.insert(0,'replicate', meta.replicate[i])
                    df.insert(df.shape[1], 'folder', inputs_folder)
                else:
                    addf = pd.DataFrame(reg_props)
                    addf.insert(0,'frame', imi+1)
                    addf.insert(0,'pos', int((os.path.basename(currd)[-2:])))
                    addf.insert(0,'replicate', meta.replicate[i])
                    addf.insert(addf.shape[1], 'folder', inputs_folder)
                    
                    df = pd.concat([df, addf])
                    del addf
                    del labels
                fails = []
            except:
                # record errors into meta file
                fails = fails + ['Folder ' + str(mainfi) + ', Pos ' + str(diri)]
        
        # end of chamber loop: save regionprops data
        df.to_csv(savedir + os.path.basename(currd) + '.csv', index = False)
        #end of position loop: update meta file
        meta = pd.read_csv(masterdir+metacsv)
        meta.loc[i, ('stardist')] = 'Done'
        if bool(fails):
            meta.loc[i, ('stardist_fails')] = fails
            
        meta.loc[i, ('stardist')] = 'Done'
        try:
            meta.to_csv(masterdir+metacsv, index = False)
            meta = pd.read_csv(masterdir+metacsv)
        except:
            print('Close meta  file!!')